## **CHICAGO 2023**

# Extracción

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Definir la URL para Boston 2023 - repositorio público AWS
url = 'https://divvy-tripdata.s3.amazonaws.com/'

# Se solicita la información a la página y usamos BeautifulSoup para obtenre un listado estructurado en XML y extraemos los archivos .zip de 2023
response = requests.get(url)
files_2023 = [
    file.text for file in BeautifulSoup(response.text, 'xml').find_all('Key')
    if file.text.endswith('.zip') and '2023' in file.text
]

print(f"Encontrados {len(files_2023)} archivos para 2023")

Encontrados 12 archivos para 2023


# Transformaciones

### Transformación 1: Filtrado de columnas y transformación 2: Creación campo "city"

In [ ]:
# T.1. TRANSFORMACIÓN DE FILTRADO: definimos el nombre del archivo CSV final y las columnas requeridas.
output_file = 'Chicago_2023.csv'
required_columns = ['started_at', 'ended_at', 'start_lat', 'start_lng', 'end_lat', 'end_lng', 'member_casual'] #se filtrará por las columnas requeridas


In [ ]:
import os
import zipfile

# Verificamos si el archivo de salida ya existe para no sobreescribirlo
if not os.path.exists(output_file):
    # Crear el archivo CSV con las columnas requeridas y la columna adicional 'city' como encabezado
    pd.DataFrame(columns=required_columns + ['city']).to_csv(output_file, index=False)

# Iteramos sobre cada archivo mensual de 2023
for file_name in files_2023:
    print(f"Procesando archivo: {file_name}")

    # Descargar el archivo y descomprimir
    file_url = url + file_name
    local_zip = file_name.split('/')[-1] #nos quedamos con el nombre del mes del archivo en lugar de toda la dirección URL

    with open(local_zip, "wb") as f: #se crea un archivo vacio para meter la información de la URL de cada mes
        f.write(requests.get(file_url).content)

    # Descomprimir el archivo en una carpeta temporal
    with zipfile.ZipFile(local_zip, "r") as zip_ref: #se abre el archivo ZIP para leerlo
        zip_ref.extractall("temp_data") #se extrae la información y se añade en la carpeta temporal

    # Cargar el CSV descomprimido y filtrar por las columnas necesarias
    csv_file = [file for file in os.listdir("temp_data") if file.endswith('.csv')][0]
    df = pd.read_csv(os.path.join("temp_data", csv_file), usecols=required_columns)

    # T.2 TRANSFORMACIÓN CREACIÓN CAMPO CIUDAD: añadir la columna 'city' con el valor "Chicago" en todas las filas
    df['city'] = 'Chicago'

    # Guardar los datos en el archivo CSV final (añadir al archivo existente)
    df.to_csv(output_file, mode='a', index=False, header=False)

    # Limpiar archivos temporales
    os.remove(local_zip)
    os.remove(os.path.join("temp_data", csv_file))

print(f"Archivo '{output_file}' actualizado con todos los meses de 2023 y la columna 'city' añadida.")

Procesando archivo: 202301-divvy-tripdata.zip
Procesando archivo: 202302-divvy-tripdata.zip
Procesando archivo: 202303-divvy-tripdata.zip
Procesando archivo: 202304-divvy-tripdata.zip
Procesando archivo: 202305-divvy-tripdata.zip
Procesando archivo: 202306-divvy-tripdata.zip
Procesando archivo: 202307-divvy-tripdata.zip
Procesando archivo: 202308-divvy-tripdata.zip
Procesando archivo: 202309-divvy-tripdata.zip
Procesando archivo: 202310-divvy-tripdata.zip
Procesando archivo: 202311-divvy-tripdata.zip
Procesando archivo: 202312-divvy-tripdata.zip
Archivo 'Chicago_2023.csv' actualizado con todos los meses de 2023 y la columna 'city' añadida.


In [ ]:
# Se imprime para comprobar el formato y la estructura
df = pd.read_csv('Chicago_2023.csv')
df

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,member_casual,city
0,2023-01-21 20:05:42,2023-01-21 20:16:33,41.924074,-87.646278,41.930000,-87.640000,member,Chicago
1,2023-01-10 15:37:36,2023-01-10 15:46:05,41.799568,-87.594747,41.809835,-87.599383,member,Chicago
2,2023-01-02 07:51:57,2023-01-02 08:05:11,42.008571,-87.690483,42.039742,-87.699413,casual,Chicago
3,2023-01-22 10:52:58,2023-01-22 11:01:44,41.799568,-87.594747,41.809835,-87.599383,member,Chicago
4,2023-01-12 13:58:01,2023-01-12 14:13:20,41.799568,-87.594747,41.809835,-87.599383,member,Chicago
...,...,...,...,...,...,...,...,...
5719872,2023-12-07 13:15:24,2023-12-07 13:17:37,41.874702,-87.649804,41.874640,-87.657030,casual,Chicago
5719873,2023-12-08 18:42:21,2023-12-08 18:45:56,41.874754,-87.649807,41.874640,-87.657030,casual,Chicago
5719874,2023-12-05 14:09:11,2023-12-05 14:13:01,41.874754,-87.649807,41.874640,-87.657030,member,Chicago
5719875,2023-12-02 21:36:07,2023-12-02 21:53:45,41.881396,-87.674984,41.885492,-87.652289,casual,Chicago


### Transformación 5: corrección formato tipo de cliente

Por último, la columna member_casual desglosa los clientes en "member" y "casual". Sin mebargo, el resto de tablas lo hacen en "Subscriber" y "Customer". Con el objetivo de homogeneizar la información, es necesario llevar a cabo otra transformación.

In [ ]:
df['member_casual'] = df['member_casual'].replace({'member': 'Subscriber', 'casual': 'Customer'})
df
df['member_casual'].head()

,member_casual
0,Subscriber
1,Subscriber
2,Customer
3,Subscriber
4,Subscriber


In [ ]:
#Descargar en csv tras las transformaciones pertinentes:
print(f"El archivo Bay Area 2023 tiene {df.shape[0]} filas y {df.shape[1]} columnas")
df.to_csv('Chicago_2023.csv', index=False)

El archivo Bay Area 2023 tiene 5719877 filas y 8 columnas
